In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import zipfile
import os
from google.colab import drive
import cv2

In [2]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Menentukan jumlah data minimum per label
min_data_per_label = 1

In [4]:
# Mendefinisikan path untuk data training
train_data_dir = '/content/drive/MyDrive/training'

In [5]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/training'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/training/mobil/1.jpg
/content/drive/MyDrive/training/mobil/2.jpg
/content/drive/MyDrive/training/mobil/3.jpg
/content/drive/MyDrive/training/mobil/4.jpg
/content/drive/MyDrive/training/mobil/5.jpg
/content/drive/MyDrive/training/mobil/6.jpg
/content/drive/MyDrive/training/mobil/7.jpg
/content/drive/MyDrive/training/mobil/8.jpeg
/content/drive/MyDrive/training/mobil/9.jpg
/content/drive/MyDrive/training/mobil/10.webp
/content/drive/MyDrive/training/mobil/11.jpeg
/content/drive/MyDrive/training/mobil/12.jpeg
/content/drive/MyDrive/training/mobil/13.jpeg
/content/drive/MyDrive/training/mobil/14.jpeg
/content/drive/MyDrive/training/mobil/15.jpeg
/content/drive/MyDrive/training/mobil/16.jpg
/content/drive/MyDrive/training/mobil/17.jpeg
/content/drive/MyDrive/training/mobil/18.jpeg
/content/drive/MyDrive/training/mobil/19.jpeg
/content/drive/MyDrive/training/mobil/20.jpeg
/content/drive/MyDrive/training/mobil/21.jpg
/content/drive/MyDrive/training/mobil/22.webp
/conten

In [6]:
 #Pra-pemrosesan dan normalisasi gambar
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Menggunakan 80% data untuk training, 20% untuk validasi
)

In [7]:
# Memuat data training dan validasi
train_generator = train_data_generator.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),  # Mengubah ukuran gambar menjadi 224x224
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_data_generator.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 382 images belonging to 2 classes.
Found 94 images belonging to 2 classes.


In [8]:
#Mengecek jumlah data per label
label_counts = np.bincount(train_generator.classes)
num_classes = len(label_counts)
assert np.min(label_counts) >= min_data_per_label, "Jumlah data per label tidak mencukupi."


In [9]:
# Memuat model VGG16 tanpa lapisan terakhir (top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 2s 0us/step


In [10]:
# Membuat model baru dengan lapisan tambahan di atas VGG16
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [11]:
# Melatih model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10, validation_data=validation_generator)


Epoch 1/10
12/12 [==============================] - 174s 12s/step - loss: 12.0843 - accuracy: 0.5052 - val_loss: 0.6893 - val_accuracy: 0.4787
Epoch 2/10
12/12 [==============================] - 5s 425ms/step - loss: 0.6882 - accuracy: 0.5105 - val_loss: 0.8246 - val_accuracy: 0.4787
Epoch 3/10
12/12 [==============================] - 5s 428ms/step - loss: 0.7272 - accuracy: 0.4843 - val_loss: 0.6662 - val_accuracy: 0.5745
Epoch 4/10
12/12 [==============================] - 5s 429ms/step - loss: 0.6738 - accuracy: 0.5942 - val_loss: 0.6299 - val_accuracy: 0.7128
Epoch 5/10
12/12 [==============================] - 5s 433ms/step - loss: 0.7105 - accuracy: 0.5707 - val_loss: 0.6472 - val_accuracy: 0.7128
Epoch 6/10
12/12 [==============================] - 5s 427ms/step - loss: 0.6325 - accuracy: 0.6230 - val_loss: 0.5577 - val_accuracy: 0.7553
Epoch 7/10
12/12 [==============================] - 5s 440ms/step - loss: 0.5418 - accuracy: 0.7068 - val_loss: 0.5227 - val_accuracy: 0.7447
Epoch

In [13]:
# Mengujikan model dengan gambar dari kamera
test_image = load_image_from_camera()  # Fungsi untuk memuat gambar dari kamera
normalized_image = test_image / 255.0  # Normalisasi gambar
reshaped_image = np.reshape(normalized_image, (1, 224, 224, 3))  # Mengubah bentuk gambar sesuai dengan input model
predictions = model.predict(reshaped_image)
predicted_label = np.argmax(predictions)


NameError: ignored

In [ ]:
# Menampilkan hasil prediksi
if predicted_label == 0:
    print("mobil")
else:
    print("motor")